In [1]:
import sys
sys.path.append('../')
from utilities import *

/home/palasso/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/palasso/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/palasso/.local/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/palasso/.local/lib/python3.7/site-packages/tensorboard/compat

# Load data

In [2]:
corpus_dir = '../../data/corpus/'
model_ngrams_dir = '../../data/ngrams/'
model_gpt2_dir = '../../data/gpt2/'

In [3]:
test = pd.read_csv(f'{corpus_dir}test.csv')

# Load results

In [4]:
testL = pd.read_csv(f'{model_ngrams_dir}testL.csv')
testKNI = pd.read_csv(f'{model_ngrams_dir}testKNI.csv')
testGPT2 = pd.read_csv(f'{model_gpt2_dir}testGPT2.csv')

In [5]:
for i in range(10):
    testGPT2[f'{i}'] = testGPT2[f'{i}'].str.lower()

# Helper function

In [6]:
gen_wer_table = lambda df: pd.DataFrame({f'{col}': [wer(correct, changed) for correct, changed in zip(test.cor, df[f'{col}'])] for col in range(10)})

In [7]:
wercor = pd.DataFrame({'0': [wer(correct, changed) for correct, changed in zip(test.cor, test.err)]})

In [8]:
wercor.mean()

0    0.21872
dtype: float64

# Lidstone (add-a smoothing) trigram

In [9]:
werL = gen_wer_table(testL)

In [10]:
werL.mean()

0    0.337290
1    0.340890
2    0.340334
3    0.334626
4    0.331666
5    0.327517
6    0.324792
7    0.323144
8    0.318380
9    0.310985
dtype: float64

In [11]:
werL.min(axis=1).mean()

0.23531882637689946

In [12]:
for i in range(10):
    print(i, sum(werL[f'{i}'] == werL.min(axis=1)))

0 611
1 489
2 428
3 465
4 463
5 483
6 493
7 556
8 617
9 776


# Trigram with Kneser-Ney smoothing with Interpolation

In [13]:
werKNI = gen_wer_table(testKNI)

In [14]:
werKNI.mean()

0    0.264579
1    0.302036
2    0.306063
3    0.307565
4    0.306975
5    0.308326
6    0.307047
7    0.304854
8    0.304088
9    0.295848
dtype: float64

In [15]:
werKNI.min(axis=1).mean()

0.222138515309207

In [16]:
for i in range(10):
    print(i, sum(werKNI[f'{i}'] == werKNI.min(axis=1)))

0 1553
1 527
2 395
3 358
4 350
5 319
6 339
7 350
8 357
9 519


# GPT-2

In [17]:
werGPT2 = gen_wer_table(testGPT2)

In [18]:
werGPT2.mean()

0    0.267053
1    0.298116
2    0.314393
3    0.321956
4    0.324894
5    0.330241
6    0.335310
7    0.339867
8    0.345723
9    0.354392
dtype: float64

In [19]:
werGPT2.min(axis=1).mean()

0.24049976211190568

In [20]:
for i in range(10):
    print(i, sum(werGPT2[f'{i}'] == werGPT2.min(axis=1)))

0 1906
1 1082
2 732
3 591
4 519
5 496
6 463
7 431
8 415
9 410


# All

In [21]:
werMIN = np.array([werL.min(axis=1), werKNI.min(axis=1), werGPT2.min(axis=1)]).min(axis=0)

In [22]:
for i in range(10):
    print(i, sum(werL[f'{i}'] == werMIN))

0 481
1 357
2 301
3 312
4 300
5 307
6 291
7 309
8 310
9 396


In [23]:
for i in range(10):
    print(i, sum(werKNI[f'{i}'] == werMIN))

0 1332
1 418
2 284
3 251
4 247
5 215
6 222
7 231
8 230
9 355


In [24]:
for i in range(10):
    print(i, sum(werGPT2[f'{i}'] == werMIN))

0 1372
1 680
2 401
3 306
4 253
5 237
6 223
7 198
8 198
9 207
